In [9]:
# import dependencies
import pandas as pd
import mysql.connector as mc
from datetime import datetime
import numpy as np
from bs4 import BeautifulSoup as Soup
import pdfkit
import win32com.client as wc
import xml.etree.ElementTree as ET
import sys

# Get db credentials
from configTest import mysql_host, mysql_u, mysql_pw, vgc_host, vgc_u, vgc_pw #, svr, db, sql_u, sql_pw, smtp_host, e_user, e_pw, port

In [10]:
# MySQL
def mysql_q (u, p, h, db, sql, cols, commit):
    # cols (0 = no, 1 = yes)
    # commit (select = 0, insert/update = 1)

    # connect to claim_qb_payments db
    cnx = mc.connect(user=u, password=p,
                    host=h,
                    database=db)
    cursor = cnx.cursor()

    # commit?
    if commit == 1:
        cursor.execute(sql)
        cnx.commit()
        sql_result = 0     
    else:
        cursor.execute(sql)
        sql_result = cursor.fetchall()

    # columns ?
    if cols == 1:
        columns=list([x[0] for x in cursor.description])
        # close connection
        cursor.close()
        cnx.close()
        # return query result [0] and columns [1]
        return sql_result
    else:
        # close connection
        cursor.close()
        cnx.close()
        # return query result
        return sql_result 

In [11]:
# Define directories
attachment_dir = 'S:/claims/letters/attachment/'
file_staging_dir = './letters/staging/'

now = datetime.now()

In [12]:
# Define file paths
attach_file = f'{attachment_dir}Claims_Paid_{now.strftime("%Y-%m-%d")}.pdf'
html_file = f'{file_staging_dir}final_rpt.html'
csv_file = f'{file_staging_dir}final_rpt.csv'
template_file = './html/pymt_summary_template.html'
html_file2 = './letters/staging/Claims_Paid.html'

In [13]:
# sql query to collect qb_txnid's
sql = '''
      SELECT rtbp_id, check_nbr, qb_txnid
      FROM ready_to_be_paid
      WHERE toVGC = 1
        AND qb_txnid <> '0';
      '''

In [14]:
# save query results as DF
qb_df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql, 0, 0))

# add column names
qb_df_cols = ['rtbp_id','check_nbr', 'qb_txnid']
qb_df.columns = qb_df_cols

In [15]:
# Connect to Quickbooks
try:
    sessionManager = wc.Dispatch("QBXMLRP2.RequestProcessor")    
    sessionManager.OpenConnection('', 'Claim Payments')
    ticket = sessionManager.BeginSession("", 2)
except Exception as e:
    print('''
    Make sure QuickBooks is running and you are logged into the Company File.
    ERROR: {}'''.format(e))
    sys.exit("Error with communicating with QuickBooks")

In [16]:
# create qbxml to query qb for check numbers
try:
    for index, row in qb_df.iterrows():
        qbxmlQuery = '''
                    <?qbxml version="14.0"?>
                    <QBXML>
                        <QBXMLMsgsRq onError="stopOnError">
                            <CheckQueryRq>
                                <TxnID>{txnId}</TxnID> 
                            </CheckQueryRq>
                        </QBXMLMsgsRq>
                    </QBXML>
                    '''.format(txnId=row['qb_txnid'])

        # Send query and receive response
        responseString = sessionManager.ProcessRequest(ticket, qbxmlQuery)

        # output Check Number (RefNumber)
        QBXML = ET.fromstring(responseString)
        QBXMLMsgsRs = QBXML.find('QBXMLMsgsRs')
        checkResults = QBXMLMsgsRs.iter("CheckRet")
        chkNbr = '0'
        for checkResult in checkResults:
            chkNbr = checkResult.find('RefNumber').text

        # Add Check Number to ready_to_be_paid table
        qb_sql_file = '''UPDATE ready_to_be_paid
                    SET check_nbr = '{ChkNbr}'
                    WHERE rtbp_id = {rowID};'''.format(ChkNbr=chkNbr, rowID=row['rtbp_id'])
        
        # execute and commit sql
        mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', qb_sql_file, 0, 1)
except Exception as e:
    print('''
    Make sure to print checks and process ACH in Quickbooks prior to starting this process.
    ERROR: {}'''.format(e))
    sys.exit("Error with communicating with QuickBooks")

In [17]:
# Disconnect from Quickbooks
sessionManager.EndSession(ticket)
sessionManager.CloseConnection()

In [18]:
# sql query for GAP claims that are RTBP
sql = '''
      SELECT rtbp_id, claim_id, claim_nbr, carrier_id, lender_name, pymt_method, first, last, pymt_type_id, amount, payment_category_id, check_nbr, qb_txnid, pymt_date, toVGC, err_msg
      FROM ready_to_be_paid
      WHERE toVGC = 1;
      '''

In [19]:
# save query results as DF
df = pd.DataFrame(mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', sql, 0, 0))

In [20]:
# add column names
df_cols = ['rtbp_id', 'claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'payment_category_id', 'check_nbr', 'qb_txnid', 'pymt_date', 'toVGC', 'err_msg']

df.columns = df_cols

In [21]:
# Add carrier name
sql = '''
    SELECT carrier_id, description
    FROM carriers;
    '''
# save query results as DF
carrier_df = pd.DataFrame(mysql_q(vgc_u, vgc_pw, vgc_host, 'visualgap_claims', sql, 0, 0))

col_names = ['carrier_id', 'carrier']
carrier_df.columns = col_names

# Merge QB_ListID into df
df = df.merge(carrier_df, left_on='carrier_id', right_on='carrier_id').copy()


In [22]:
# format df
# list of conditions
type_conds = [((df['payment_category_id'] == 1) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 1) & (df['pymt_type_id'] == 2)),
              ((df['payment_category_id'] == 2) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 2) & (df['pymt_type_id'] == 2)),
              ((df['payment_category_id'] == 3) & (df['pymt_type_id'] == 1)),
              ((df['payment_category_id'] == 3) & (df['pymt_type_id'] == 2)),]
# list of name types
type_name = ['GAP', 'GAP Supp', 'GAP Plus', 'GAP Plus Supp', 'TotalRestart', 'TotalRestart Supp']

# add column and assigned values
df['Claim_Type'] = np.select(type_conds, type_name)


In [23]:
# Create df to update VGC
vgc_update_df = df[['claim_id', 'pymt_method', 'pymt_type_id', 'amount', 'payment_category_id', 'check_nbr', 'qb_txnid', 'pymt_date']].copy()
vgc_update_df

,claim_id,pymt_method,pymt_type_id,amount,payment_category_id,check_nbr,qb_txnid,pymt_date
0,1466,Check,1,5302.64,1,11219,899-1644530793,2022-02-10
1,1471,ACH-GL,1,0.00,1,0,0,2022-02-10
2,1472,ACH-GL,1,5138.04,1,ACH,89C-1644530800,2022-02-10
3,1475,ACH-GL,1,653.87,1,ACH,89F-1644530807,2022-02-10
4,1476,ACH-GL,1,0.00,1,0,0,2022-02-10
5,1474,Check,1,0.00,1,0,0,2022-02-10
6,1477,Check,2,191.17,1,11220,8A2-1644530816,2022-02-10
7,1488,Check,1,3498.00,1,11221,8A5-1644530825,2022-02-10
8,1490,ACH-GL,1,0.00,1,0,0,2022-02-10
9,1475,ACH-GL,1,1000.00,2,ACH,8A8-1644530832,2022-02-10


In [24]:
# Create df for Claim Payment Summary
final_rpt_df = df[['claim_nbr', 'carrier', 'lender_name', 'first', 'last', 'amount', 'pymt_method', 'check_nbr', 'Claim_Type']].copy()

In [25]:
# Format df
final_rpt_df.rename(columns = {'claim_nbr':'Claim Nbr', 'carrier':'Carrier', 'alt_name':'Lender', 'first':'First Name', 'last':'Last Name', 'amount':'Amount', 'pymt_method':'Method',
                               'check_nbr':'Check Nbr', 'Claim_Type':'Claim Type'}, inplace=True)
                               
final_rpt_df['Amount'] = final_rpt_df['Amount'].map('${:,.2f}'.format)

In [26]:
final_rpt_df

,Claim Nbr,Carrier,lender_name,First Name,Last Name,Amount,Method,Check Nbr,Claim Type
0,202109172488,ANICO,Charlotte Metro Credit Union,Sam,Hunt,"$5,302.64",Check,11219,GAP
1,202110182493,ANICO,Charlotte Metro Credit Union,ANDREW,FLYNN,$0.00,ACH-GL,0,GAP
2,202110182494,ANICO,Charlotte Metro Credit Union,TYSHAUN,GRANT,"$5,138.04",ACH-GL,ACH,GAP
3,202110192497,ANICO,Charlotte Metro Credit Union,OCTAVIO,PARTIDA,$653.87,ACH-GL,ACH,GAP
4,202110192498,ANICO,Charlotte Metro Credit Union,SHERRY,VOGT,$0.00,ACH-GL,0,GAP
5,202110192496,ANICO,City Credit Union,JOSHUA,WELCH,$0.00,Check,0,GAP
6,202110192499,ANICO,City Credit Union,BRIAN,CHANDLER,$191.17,Check,11220,GAP Supp
7,202110272510,ANICO,City Credit Union,TYSHAUN,GRANT,"$3,498.00",Check,11221,GAP
8,202111092512,ANICO,Charlotte Metro Credit Union,Ralph,Sutton,$0.00,ACH-GL,0,GAP
9,202110192497,ANICO,Charlotte Metro Credit Union,OCTAVIO,PARTIDA,"$1,000.00",ACH-GL,ACH,GAP Plus


In [27]:
# export is df as csv
final_rpt_df.to_csv(csv_file, index=False)
# read in csv
csvFile = pd.read_csv(csv_file)
# convert csv to html
csvFile.to_html(html_file, index=False)

In [28]:
# Get html df
soup = Soup(open(html_file), "html.parser")
table = str(soup.select_one("table", {"class":"dataframe"}))
# Get template
soup2 = Soup(open(template_file), "html.parser")
df_div = soup2.find("div", {"id":"df"})
# append table to template
df_div.append(Soup(table, 'html.parser'))
# write html file
with open(html_file2,'w') as file:
    file.write(str(soup2))

In [29]:
# create PDF from html
pdf_options = {'orientation': 'landscape',
                'page-size': 'Letter',
                'margin-top': '0.25in',
                'margin-right': '0.25in',
                'margin-bottom': '0.25in',
                'margin-left': '0.25in',
                'encoding': "UTF-8",}
pdfkit.from_file(html_file2, attach_file, options=pdf_options)

True

In [30]:
# Email Claim Summary Report

In [31]:
# update toVGC to 3
if len(df) > 0:
    for index, row in df.iterrows():
        err_sql = '''
                UPDATE ready_to_be_paid
                SET toVGC = 3
                WHERE rtbp_id = {rtbp_id};
                '''.format(rtbp_id=row['rtbp_id'])
        # run update query        
        mysql_q(mysql_u, mysql_pw, mysql_host, 'claim_qb_payments', err_sql, 0, 1)